In [ ]:
## Period Validation

After BLS finds a period, we should validate it by checking for:
1. **Period aliases**: Is the reported period a multiple/fraction of the true period? (2×, 3×, 4×, 5×, 0.5×)
2. **Single-transit events**: Do we only have one transit? If so, the period is not well constrained.

The validation works by computing SNR for different period hypotheses and optimizing the epoch (t0) for each.

In [ ]:
# Import the period validation module
from pytfit5 import period_validation as pval
import pytfit5.keplerian as kep

In [ ]:
# Validate the BLS result from the synthetic data
# This will test period factors: 0.5×, 1.0×, 2.0×, 3.0×, 4.0×, 5.0×
results, transit_info = pval.validate_bls_period(
    time=phot_syn.time,
    flux=phot_syn.flux_f,
    bls_result=syn_ans,
    test_factors=[0.5, 1.0, 2.0, 3.0, 4.0, 5.0],
    optimize_t0=True,
    verbose=False
)



In [ ]:
# After running period validation, update BLS results if needed
# ================================================================

# Check if validation found a better period (alias detection or single-transit)
if results['is_alias']:
    # Period alias detected - use the corrected period
    print(f"Period alias detected! Updating from {syn_ans.bper:.6f} to {results['best_period']:.6f} days")
    
    # Update BLS result object with validated values
    syn_ans.bper = results['best_period']  # Use validated period
    syn_ans.epo = results['best_t0']        # Use optimized epoch
    
    print(f"Updated BLS results:")
    print(f"  Period: {syn_ans.bper:.6f} days")
    print(f"  Epoch (t0): {syn_ans.epo:.6f} days")
    print(f"  SNR improvement: {results['snr_improvement']:.2f}")
    
elif results['is_single_transit']:
    # Single transit - period is poorly constrained
    print(f"Single transit detected - using period estimate")
    
    # Find the transit with the highest SNR (most reliable detection)
    max_snr_idx = np.argmax(transit_info['transit_snrs'])
    single_transit_time = transit_info['transit_times'][max_snr_idx]
    single_transit_snr = transit_info['transit_snrs'][max_snr_idx]
    
    print(f"  Using transit with highest SNR: {single_transit_snr:.2f}")
    print(f"  Transit time: {single_transit_time:.6f} days (index {max_snr_idx})")
    
    # Estimate period using stellar density and the dominant transit time
    P_est, P_min, is_constrained, n_transits = pval.estimate_single_transit_period(
        time=phot_syn.time,
        tdur=syn_ans.tdur,
        rho_star=pval.get_stellar_density(syn_inputs.mstar, syn_inputs.rstar),
        margin_factor=1.5
    )
    
    print(f"  Estimated period: {P_est:.2f} days")
    print(f"  Minimum period (data-constrained): {P_min:.2f} days")
    print(f"  Expected transits: {n_transits:.2f}")
    
    # Update BLS results with estimated period and highest-SNR transit time
    syn_ans.bper = P_est
    syn_ans.epo = single_transit_time  # Use the highest SNR transit as epoch
    syn_ans.snr = single_transit_snr   # Update with single transit SNR
    
    print(f"  Using highest-SNR transit time as epoch: {syn_ans.epo:.6f} days")
    
    # Note: For single transits, the period is uncertain
    # You may want to keep the original BLS period or use the estimate
    # The epoch should be the time of the single observed transit
    
else:
    # BLS period confirmed - but still use optimized epoch
    print(f"BLS period confirmed: {syn_ans.bper:.6f} days")
    
    # Update epoch to the optimized value (better SNR)
    if results['best_t0'] != syn_ans.epo:
        print(f"Updating epoch from {syn_ans.epo:.6f} to {results['best_t0']:.6f} days")
        syn_ans.epo = results['best_t0']

# print(f"\nFinal BLS results:")
# print(f"  Period: {syn_ans.bper:.6f} days")
# print(f"  Epoch: {syn_ans.epo:.6f} days")
# print(f"  Depth: {syn_ans.depth:.6f} ({syn_ans.depth*1e6:.1f} ppm)")
# print(f"  Duration: {syn_ans.tdur:.6f} days ({syn_ans.tdur*24:.2f} hours)")
# print(f"  SNR: {syn_ans.snr:.2f}")